In [2]:
from datasets import load_dataset

ds = load_dataset("huyhuy123/ViLQA")

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'ID1', 'Question', 'Answer', 'idx'],
        num_rows: 43588
    })
})

In [4]:
# Phân tích cấu trúc dữ liệu ViLQA dataset
import pandas as pd
import numpy as np

print("📊 Thông tin dataset ViLQA:")
print(f"- Total samples: {len(ds['train'])} mẫu")
print(f"- Features: {list(ds['train'].features.keys())}")

print("\n🔍 Xem mẫu dữ liệu đầu tiên:")
sample = ds['train'][0]
for key, value in sample.items():
    if isinstance(value, str) and len(value) > 100:
        print(f"{key}: {value[:100]}...")
    else:
        print(f"{key}: {value}")

print("\n📝 Một vài mẫu khác:")
for i in range(1, 4):
    sample = ds['train'][i]
    print(f"\nSample {i+1}:")
    print(f"Question: {sample['Question'][:80]}...")
    print(f"Answer: {sample['Answer'][:80]}...")

📊 Thông tin dataset ViLQA:
- Total samples: 43588 mẫu
- Features: ['ID', 'ID1', 'Question', 'Answer', 'idx']

🔍 Xem mẫu dữ liệu đầu tiên:
ID: 29
ID1: 2
Question: Thực hiện cắt giảm hồ sơ thay đổi mức vốn điều lệ của ngân hàng hợp tác xã trong quý 2 năm 2024?
Answer: Căn cứ Mục 3 Phần 3 Phương án cắt giảm, đơn giản hóa quy định liên quan đến hoạt động kinh doanh thu...
idx: 0

📝 Một vài mẫu khác:

Sample 2:
Question: Mẫu đơn đề nghị chấp thuận thay đổi mức vốn điều lệ của ngân hàng hợp tác xã mới...
Answer: Đơn đề nghị chấp thuận thay đổi mức vốn điều lệ của ngân hàng hợp tác xã hiện na...

Sample 3:
Question: Thỏa thuận cho vay giữa khách hàng và ngân hàng có phải lập thành văn bản không?...
Answer: Căn cứ theo khoản 1, khoản 2 Điều 23 Thông tư 39/2016/TT-NHNN, thỏa thuận cho va...

Sample 4:
Question: Khách hàng có thể vay ngân hàng theo hình thức nào?...
Answer: Căn cứ theo Điều 10 Thông tư 39/2016/TT-NHNN quy định như sau: Như vậy, khách hà...


In [5]:
# Phân tích chất lượng dữ liệu Question và Answer
def analyze_vilqa_quality(dataset):
    """Phân tích chất lượng dữ liệu ViLQA"""
    questions = [item['Question'] for item in dataset]
    answers = [item['Answer'] for item in dataset]
    
    print(f"📈 Phân tích Dataset ViLQA ({len(dataset)} samples):")
    
    # Độ dài câu hỏi
    question_lengths = [len(q) if q else 0 for q in questions]
    print(f"🔸 Độ dài câu hỏi:")
    print(f"   - Trung bình: {np.mean(question_lengths):.1f} ký tự")
    print(f"   - Min: {min(question_lengths)}, Max: {max(question_lengths)}")
    print(f"   - Median: {np.median(question_lengths):.1f}")
    
    # Độ dài câu trả lời
    answer_lengths = [len(a) if a else 0 for a in answers]
    print(f"🔸 Độ dài câu trả lời:")
    print(f"   - Trung bình: {np.mean(answer_lengths):.1f} ký tự")
    print(f"   - Min: {min(answer_lengths)}, Max: {max(answer_lengths)}")
    print(f"   - Median: {np.median(answer_lengths):.1f}")
    
    # Kiểm tra dữ liệu rỗng
    empty_questions = sum(1 for q in questions if not q or q.strip() == "")
    empty_answers = sum(1 for a in answers if not a or a.strip() == "")
    
    print(f"🔸 Dữ liệu rỗng:")
    print(f"   - Câu hỏi rỗng: {empty_questions}")
    print(f"   - Câu trả lời rỗng: {empty_answers}")
    
    # Phân tích câu hỏi kết thúc bằng dấu ?
    questions_with_qmark = sum(1 for q in questions if q and q.strip().endswith('?'))
    print(f"🔸 Câu hỏi có dấu '?': {questions_with_qmark}/{len(questions)} ({questions_with_qmark/len(questions)*100:.1f}%)")
    
    return {
        'questions': questions,
        'answers': answers,
        'question_lengths': question_lengths,
        'answer_lengths': answer_lengths,
        'empty_questions': empty_questions,
        'empty_answers': empty_answers,
        'questions_with_qmark': questions_with_qmark
    }

# Phân tích dataset ViLQA
vilqa_analysis = analyze_vilqa_quality(ds['train'])

📈 Phân tích Dataset ViLQA (43588 samples):
🔸 Độ dài câu hỏi:
   - Trung bình: 75.7 ký tự
   - Min: 0, Max: 263
   - Median: 71.0
🔸 Độ dài câu trả lời:
   - Trung bình: 888.6 ký tự
   - Min: 0, Max: 20674
   - Median: 673.0
🔸 Dữ liệu rỗng:
   - Câu hỏi rỗng: 48
   - Câu trả lời rỗng: 115
🔸 Câu hỏi có dấu '?': 42502/43588 (97.5%)


In [6]:
# Làm sạch và xử lý dữ liệu ViLQA
def clean_text(text):
    """Làm sạch text"""
    if not text:
        return ""
    
    # Loại bỏ khoảng trắng thừa
    text = " ".join(text.split())
    
    # Loại bỏ ký tự đặc biệt không cần thiết
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    
    return text.strip()

def process_vilqa_dataset(dataset, max_answer_length=8000):
    """Xử lý dataset ViLQA và lọc dữ liệu chất lượng"""
    processed_data = []
    
    for item in dataset:
        # Lấy dữ liệu từ các trường đúng
        question = clean_text(item.get('Question', ''))
        answer = clean_text(item.get('Answer', ''))
        
        # Lọc dữ liệu theo tiêu chí chất lượng
        if (len(question) >= 10 and  # Câu hỏi ít nhất 10 ký tự
            len(answer) >= 50 and    # Câu trả lời ít nhất 50 ký tự
            len(answer) <= max_answer_length):  # Giới hạn độ dài trả lời
            
            processed_data.append({
                'question': question,
                'answer': answer
            })
    
    return processed_data

print("🔧 Đang xử lý và làm sạch dữ liệu ViLQA...")

# Xử lý dataset
vilqa_processed = process_vilqa_dataset(ds['train'])
print(f"✅ Dataset processed: {len(ds['train'])} → {len(vilqa_processed)} (giữ lại {len(vilqa_processed)/len(ds['train'])*100:.1f}%)")

print(f"\n📊 Thống kê sau xử lý:")
print(f"- Tổng số mẫu chất lượng: {len(vilqa_processed)}")

# Phân tích lại sau khi xử lý
if vilqa_processed:
    processed_q_lengths = [len(item['question']) for item in vilqa_processed]
    processed_a_lengths = [len(item['answer']) for item in vilqa_processed]
    
    print(f"\n🔸 Độ dài câu hỏi sau xử lý:")
    print(f"   - Trung bình: {np.mean(processed_q_lengths):.1f} ký tự")
    print(f"   - Min: {min(processed_q_lengths)}, Max: {max(processed_q_lengths)}")
    
    print(f"🔸 Độ dài câu trả lời sau xử lý:")
    print(f"   - Trung bình: {np.mean(processed_a_lengths):.1f} ký tự")
    print(f"   - Min: {min(processed_a_lengths)}, Max: {max(processed_a_lengths)}")

🔧 Đang xử lý và làm sạch dữ liệu ViLQA...
✅ Dataset processed: 43588 → 43420 (giữ lại 99.6%)

📊 Thống kê sau xử lý:
- Tổng số mẫu chất lượng: 43420

🔸 Độ dài câu hỏi sau xử lý:
   - Trung bình: 75.9 ký tự
   - Min: 10, Max: 263
🔸 Độ dài câu trả lời sau xử lý:
   - Trung bình: 882.7 ký tự
   - Min: 51, Max: 7981
✅ Dataset processed: 43588 → 43420 (giữ lại 99.6%)

📊 Thống kê sau xử lý:
- Tổng số mẫu chất lượng: 43420

🔸 Độ dài câu hỏi sau xử lý:
   - Trung bình: 75.9 ký tự
   - Min: 10, Max: 263
🔸 Độ dài câu trả lời sau xử lý:
   - Trung bình: 882.7 ký tự
   - Min: 51, Max: 7981


In [9]:
# Tạo thư mục và lưu dữ liệu ViLQA
import os
import json

# Tạo thư mục data/finetune_data cho ViLQA
output_dir = "../data/finetune_data2"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Sử dụng thư mục: {output_dir}")

# Lưu dữ liệu dưới nhiều format khác nhau
def save_jsonl(data, filepath):
    """Lưu dữ liệu dưới định dạng JSONL"""
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

def save_instruction_format(data, filepath):
    """Lưu dữ liệu dưới định dạng instruction tuning"""
    instruction_data = []
    for item in data:
        instruction_item = {
            "instruction": "Trả lời câu hỏi pháp luật sau:",
            "input": item['question'],
            "output": item['answer']
        }
        instruction_data.append(instruction_item)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in instruction_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

def save_conversation_format(data, filepath):
    """Lưu dữ liệu dưới định dạng conversation"""
    conversation_data = []
    for item in data:
        conversation_item = {
            "conversations": [
                {"role": "user", "content": item['question']},
                {"role": "assistant", "content": item['answer']}
            ]
        }
        conversation_data.append(conversation_item)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in conversation_data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

print("💾 Đang lưu dữ liệu ViLQA...")

# Lưu QA format
vilqa_qa_path = os.path.join(output_dir, "vilqa_qa_format.jsonl")
save_jsonl(vilqa_processed, vilqa_qa_path)
print(f"✅ Đã lưu ViLQA QA format: {vilqa_qa_path}")

# Lưu instruction format
vilqa_instruction_path = os.path.join(output_dir, "vilqa_instruction_format.jsonl")
save_instruction_format(vilqa_processed, vilqa_instruction_path)
print(f"✅ Đã lưu ViLQA instruction format: {vilqa_instruction_path}")

# Lưu conversation format
vilqa_conversation_path = os.path.join(output_dir, "vilqa_conversation_format.jsonl")
save_conversation_format(vilqa_processed, vilqa_conversation_path)
print(f"✅ Đã lưu ViLQA conversation format: {vilqa_conversation_path}")

print(f"\n📊 Tổng kết:")
print(f"- Tổng số mẫu training: {len(vilqa_processed)}")
print(f"- Format: QA, Instruction, Conversation")

📁 Sử dụng thư mục: ../data/finetune_data2
💾 Đang lưu dữ liệu ViLQA...
✅ Đã lưu ViLQA QA format: ../data/finetune_data2\vilqa_qa_format.jsonl
✅ Đã lưu ViLQA QA format: ../data/finetune_data2\vilqa_qa_format.jsonl
✅ Đã lưu ViLQA instruction format: ../data/finetune_data2\vilqa_instruction_format.jsonl
✅ Đã lưu ViLQA instruction format: ../data/finetune_data2\vilqa_instruction_format.jsonl
✅ Đã lưu ViLQA conversation format: ../data/finetune_data2\vilqa_conversation_format.jsonl

📊 Tổng kết:
- Tổng số mẫu training: 43420
- Format: QA, Instruction, Conversation
✅ Đã lưu ViLQA conversation format: ../data/finetune_data2\vilqa_conversation_format.jsonl

📊 Tổng kết:
- Tổng số mẫu training: 43420
- Format: QA, Instruction, Conversation


In [10]:
# Tạo metadata cho ViLQA dataset
vilqa_metadata = {
    "dataset_info": {
        "source": "huyhuy123/ViLQA",
        "description": "Vietnamese Legal Q&A Dataset (ViLQA) processed for fine-tuning",
        "total_samples": len(vilqa_processed),
        "training_samples": len(vilqa_processed),
        "split_type": "training_only"
    },
    "processing_info": {
        "filters_applied": [
            "Minimum question length: 10 characters",
            "Minimum answer length: 50 characters", 
            "Maximum answer length: 8000 characters",
            "Text cleaning: removed extra whitespace and special characters"
        ],
        "retention_rate": f"{len(vilqa_processed)/len(ds['train'])*100:.1f}%",
        "original_samples": len(ds['train']),
        "processed_samples": len(vilqa_processed)
    },
    "file_formats": {
        "qa_format": "Simple question-answer pairs for training",
        "instruction_format": "Instruction tuning format with instruction/input/output",
        "conversation_format": "Multi-turn conversation format for chatbot training"
    },
    "statistics": {
        "avg_question_length": np.mean([len(item['question']) for item in vilqa_processed]),
        "avg_answer_length": np.mean([len(item['answer']) for item in vilqa_processed])
    }
}

# Lưu metadata cho ViLQA
vilqa_metadata_path = os.path.join(output_dir, "vilqa_metadata.json")
with open(vilqa_metadata_path, 'w', encoding='utf-8') as f:
    json.dump(vilqa_metadata, f, indent=2, ensure_ascii=False)

print(f"✅ Đã lưu ViLQA metadata: {vilqa_metadata_path}")

# Hiển thị mẫu dữ liệu từ các format
print("\n📝 Mẫu dữ liệu ViLQA các format:")

print("\n🔸 QA Format:")
sample_qa = vilqa_processed[0]
print(f"Question: {sample_qa['question'][:100]}...")
print(f"Answer: {sample_qa['answer'][:100]}...")

print("\n🔸 Instruction Format:")
print(f"Instruction: Trả lời câu hỏi pháp luật sau:")
print(f"Input: {sample_qa['question'][:100]}...")
print(f"Output: {sample_qa['answer'][:100]}...")

print("\n🔸 Conversation Format:")
print(f"User: {sample_qa['question'][:100]}...")
print(f"Assistant: {sample_qa['answer'][:100]}...")

print(f"\n🎉 Hoàn thành xử lý ViLQA! Đã lưu {len(vilqa_processed)} mẫu dữ liệu training vào {output_dir}")

# Hiển thị thống kê file
print(f"\n📁 Files ViLQA đã tạo:")
vilqa_files = ["vilqa_qa_format.jsonl", "vilqa_instruction_format.jsonl", "vilqa_conversation_format.jsonl", "vilqa_metadata.json"]
for filename in vilqa_files:
    filepath = os.path.join(output_dir, filename)
    if os.path.exists(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"   - {filename}: {size_mb:.2f} MB")

✅ Đã lưu ViLQA metadata: ../data/finetune_data2\vilqa_metadata.json

📝 Mẫu dữ liệu ViLQA các format:

🔸 QA Format:
Question: Thực hiện cắt giảm hồ sơ thay đổi mức vốn điều lệ của ngân hàng hợp tác xã trong quý 2 năm 2024?...
Answer: Căn cứ Mục 3 Phần 3 Phương án cắt giảm, đơn giản hóa quy định liên quan đến hoạt động kinh doanh thu...

🔸 Instruction Format:
Instruction: Trả lời câu hỏi pháp luật sau:
Input: Thực hiện cắt giảm hồ sơ thay đổi mức vốn điều lệ của ngân hàng hợp tác xã trong quý 2 năm 2024?...
Output: Căn cứ Mục 3 Phần 3 Phương án cắt giảm, đơn giản hóa quy định liên quan đến hoạt động kinh doanh thu...

🔸 Conversation Format:
User: Thực hiện cắt giảm hồ sơ thay đổi mức vốn điều lệ của ngân hàng hợp tác xã trong quý 2 năm 2024?...
Assistant: Căn cứ Mục 3 Phần 3 Phương án cắt giảm, đơn giản hóa quy định liên quan đến hoạt động kinh doanh thu...

🎉 Hoàn thành xử lý ViLQA! Đã lưu 43420 mẫu dữ liệu training vào ../data/finetune_data2

📁 Files ViLQA đã tạo:
   - vilqa_qa_format.

In [ ]:
# Validate dữ liệu ViLQA đã lưu
def validate_jsonl_file(filepath, expected_count, dataset_name=""):
    """Kiểm tra tính toàn vẹn của file JSONL"""
    try:
        count = 0
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                json.loads(line.strip())  # Kiểm tra JSON hợp lệ
                count += 1
        
        if count == expected_count:
            print(f"✅ {os.path.basename(filepath)}: {count} dòng (OK)")
            return True
        else:
            print(f"❌ {os.path.basename(filepath)}: {count} dòng (Expected: {expected_count})")
            return False
    except Exception as e:
        print(f"❌ Error validating {os.path.basename(filepath)}: {e}")
        return False

print("🔍 Kiểm tra tính toàn vẹn dữ liệu ViLQA:")

vilqa_files_to_check = [
    "vilqa_qa_format.jsonl",
    "vilqa_instruction_format.jsonl", 
    "vilqa_conversation_format.jsonl"
]

all_valid = True

print("\n📂 ViLQA files:")
for filename in vilqa_files_to_check:
    filepath = os.path.join(output_dir, filename)
    if not validate_jsonl_file(filepath, len(vilqa_processed), "ViLQA"):
        all_valid = False

# Kiểm tra metadata
vilqa_metadata_file = os.path.join(output_dir, "vilqa_metadata.json")
try:
    with open(vilqa_metadata_file, 'r', encoding='utf-8') as f:
        metadata_loaded = json.load(f)
    print(f"✅ vilqa_metadata.json: OK")
except Exception as e:
    print(f"❌ vilqa_metadata.json: Error - {e}")
    all_valid = False

if all_valid:
    print(f"\n🎉 Tất cả files ViLQA đều hợp lệ! Dataset đã sẵn sàng để training.")
    print(f"\n💡 Sử dụng cho training:")
    print(f"   - vilqa_qa_format.jsonl: Cho traditional Q&A training")
    print(f"   - vilqa_instruction_format.jsonl: Cho instruction-following models") 
    print(f"   - vilqa_conversation_format.jsonl: Cho chatbot training")
    print(f"   - Tổng {len(vilqa_processed)} mẫu training data")
else:
    print(f"\n⚠️ Có lỗi xảy ra với files ViLQA. Vui lòng kiểm tra lại.")

print(f"\n📍 Đường dẫn dữ liệu ViLQA: {os.path.abspath(output_dir)}")

# Tổng kết final
print(f"\n📋 Tổng kết dataset ViLQA:")
print(f"   - Nguồn: huyhuy123/ViLQA")
print(f"   - Số mẫu gốc: {len(ds['train'])}")
print(f"   - Số mẫu sau xử lý: {len(vilqa_processed)}")
print(f"   - Tỷ lệ giữ lại: {len(vilqa_processed)/len(ds['train'])*100:.1f}%")
print(f"   - Mục đích: Training chatbot pháp luật")